## Introduction 

Cette leçon couvrira : 
- Qu'est-ce que l'appel de fonction et ses cas d'utilisation 
- Comment créer un appel de fonction en utilisant OpenAI 
- Comment intégrer un appel de fonction dans une application 

## Objectifs d'apprentissage 

Après avoir terminé cette leçon, vous saurez comment faire et comprendre : 

- Le but de l'utilisation de l'appel de fonction 
- Configurer un appel de fonction en utilisant le service OpenAI 
- Concevoir des appels de fonction efficaces pour le cas d'utilisation de votre application


## Comprendre les appels de fonction

Pour cette leçon, nous voulons créer une fonctionnalité pour notre startup éducative qui permet aux utilisateurs d'utiliser un chatbot pour trouver des cours techniques. Nous recommanderons des cours adaptés à leur niveau de compétence, leur rôle actuel et la technologie qui les intéresse.

Pour cela, nous utiliserons une combinaison de :
 - `OpenAI` pour créer une expérience de chat pour l'utilisateur
 - `Microsoft Learn Catalog API` pour aider les utilisateurs à trouver des cours en fonction de leur demande
 - `Function Calling` pour prendre la requête de l'utilisateur et l'envoyer à une fonction afin de faire la requête API.

Pour commencer, voyons pourquoi nous voudrions utiliser les appels de fonction en premier lieu :

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obtenir une nouvelle réponse de GPT où il peut voir la réponse de la fonction


print(second_response.choices[0].message)


### Pourquoi l'appel de fonction

Si vous avez suivi une autre leçon de ce cours, vous comprenez probablement la puissance de l'utilisation des grands modèles de langage (LLM). Espérons que vous pouvez également voir certaines de leurs limites.

L'appel de fonction est une fonctionnalité du service OpenAI conçue pour répondre aux défis suivants :

Formatage incohérent des réponses :
- Avant l'appel de fonction, les réponses d'un grand modèle de langage étaient non structurées et incohérentes. Les développeurs devaient écrire un code de validation complexe pour gérer chaque variation dans la sortie.

Intégration limitée avec des données externes :
- Avant cette fonctionnalité, il était difficile d'incorporer des données provenant d'autres parties d'une application dans un contexte de chat.

En standardisant les formats de réponse et en permettant une intégration transparente avec des données externes, l'appel de fonction simplifie le développement et réduit le besoin de logique de validation supplémentaire.

Les utilisateurs ne pouvaient pas obtenir de réponses comme « Quel temps fait-il actuellement à Stockholm ? ». Cela est dû au fait que les modèles étaient limités à la période sur laquelle les données avaient été entraînées.

Regardons l'exemple ci-dessous qui illustre ce problème :

Disons que nous voulons créer une base de données de données d'étudiants afin de pouvoir leur suggérer le bon cours. Ci-dessous, nous avons deux descriptions d'étudiants qui sont très similaires dans les données qu'elles contiennent.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Nous voulons envoyer ceci à un LLM pour analyser les données. Cela pourra ensuite être utilisé dans notre application pour envoyer cela à une API ou le stocker dans une base de données.

Créons deux invites identiques dans lesquelles nous indiquons au LLM quelles informations nous intéressent :


Nous voulons envoyer ceci à un LLM pour analyser les parties importantes pour notre produit. Ainsi, nous pouvons créer deux invites identiques pour instruire le LLM :


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Après avoir créé ces deux invites, nous les enverrons au LLM en utilisant `openai.ChatCompletion`. Nous stockons l'invite dans la variable `messages` et attribuons le rôle à `user`. Cela permet d'imiter un message d'un utilisateur écrit à un chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nous pouvons maintenant envoyer les deux requêtes au LLM et examiner la réponse que nous recevons.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Même si les invites sont les mêmes et que les descriptions sont similaires, nous pouvons obtenir différents formats pour la propriété `Grades`.

Si vous exécutez la cellule ci-dessus plusieurs fois, le format peut être `3.7` ou `3.7 GPA`.

Cela s'explique par le fait que le LLM prend des données non structurées sous forme d'invite écrite et renvoie également des données non structurées. Nous devons disposer d'un format structuré afin de savoir à quoi nous attendre lors du stockage ou de l'utilisation de ces données.

En utilisant l'appel fonctionnel, nous pouvons nous assurer de recevoir des données structurées en retour. Lors de l'utilisation de l'appel fonctionnel, le LLM n'appelle ni n'exécute réellement aucune fonction. Au lieu de cela, nous créons une structure que le LLM doit suivre pour ses réponses. Nous utilisons ensuite ces réponses structurées pour savoir quelle fonction exécuter dans nos applications.


![Diagramme de flux d'appel de fonction](../../../../translated_images/Function-Flow.083875364af4f4bb.fr.png)


Nous pouvons ensuite prendre ce qui est retourné par la fonction et le renvoyer au LLM. Le LLM répondra alors en utilisant un langage naturel pour répondre à la requête de l'utilisateur.


### Cas d'utilisation pour l'utilisation des appels de fonction

**Appeler des outils externes**  
Les chatbots sont excellents pour fournir des réponses aux questions des utilisateurs. En utilisant les appels de fonction, les chatbots peuvent utiliser les messages des utilisateurs pour accomplir certaines tâches. Par exemple, un étudiant peut demander au chatbot « Envoyer un e-mail à mon instructeur en disant que j'ai besoin de plus d'aide sur ce sujet ». Cela peut déclencher un appel de fonction à `send_email(to: string, body: string)`


**Créer des requêtes API ou base de données**  
Les utilisateurs peuvent trouver des informations en utilisant un langage naturel qui est converti en une requête formatée ou une requête API. Un exemple pourrait être un enseignant qui demande « Qui sont les étudiants qui ont terminé le dernier devoir » ce qui pourrait appeler une fonction nommée `get_completed(student_name: string, assignment: int, current_status: string)`


**Créer des données structurées**  
Les utilisateurs peuvent prendre un bloc de texte ou un CSV et utiliser le LLM pour en extraire des informations importantes. Par exemple, un étudiant peut convertir un article Wikipédia sur les accords de paix pour créer des fiches de révision IA. Cela peut être fait en utilisant une fonction appelée `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Création de votre premier appel de fonction

Le processus de création d'un appel de fonction comprend 3 étapes principales :  
1. Appeler l'API Chat Completions avec une liste de vos fonctions et un message utilisateur  
2. Lire la réponse du modèle pour effectuer une action, c'est-à-dire exécuter une fonction ou un appel d'API  
3. Faire un autre appel à l'API Chat Completions avec la réponse de votre fonction pour utiliser cette information afin de créer une réponse pour l'utilisateur.


![Flux d'un appel de fonction](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.fr.png)


### Éléments d'un appel de fonction 

#### Entrée des utilisateurs 

La première étape consiste à créer un message utilisateur. Cela peut être attribué dynamiquement en prenant la valeur d'une entrée texte ou vous pouvez attribuer une valeur ici. Si c'est votre première fois à travailler avec l'API Chat Completions, nous devons définir le `role` et le `content` du message. 

Le `role` peut être soit `system` (création de règles), `assistant` (le modèle) ou `user` (l'utilisateur final). Pour l'appel de fonction, nous allons l'assigner en tant que `user` avec une question d'exemple. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Création de fonctions.

Ensuite, nous allons définir une fonction et les paramètres de cette fonction. Nous utiliserons ici une seule fonction appelée `search_courses`, mais vous pouvez en créer plusieurs.

**Important** : Les fonctions sont incluses dans le message système destiné au LLM et seront comptabilisées dans le nombre de tokens disponibles.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Définitions** 

La structure de définition de fonction comporte plusieurs niveaux, chacun avec ses propres propriétés. Voici une répartition de la structure imbriquée :

**Propriétés de fonction de niveau supérieur :**

`name` - Le nom de la fonction que nous souhaitons appeler. 

`description` - C'est la description du fonctionnement de la fonction. Il est important ici d'être précis et clair. 

`parameters` - Une liste de valeurs et de formats que vous souhaitez que le modèle produise dans sa réponse. 

**Propriétés de l'objet Parameters :**

`type` - Le type de données de l'objet parameters (généralement "object")

`properties` - Liste des valeurs spécifiques que le modèle utilisera pour sa réponse. 

**Propriétés des paramètres individuels :**

`name` - Défini implicitement par la clé de la propriété (par exemple, "role", "product", "level")

`type` - Le type de données de ce paramètre spécifique (par exemple, "string", "number", "boolean") 

`description` - Description du paramètre spécifique. 

**Propriétés optionnelles :**

`required` - Un tableau listant les paramètres requis pour que l'appel de fonction soit complété. 


### Appeler la fonction  
Après avoir défini une fonction, nous devons maintenant l’inclure dans l’appel à l’API Chat Completion. Nous le faisons en ajoutant `functions` à la requête. Dans ce cas, `functions=functions`.  

Il y a aussi une option pour définir `function_call` sur `auto`. Cela signifie que nous laisserons le LLM décider quelle fonction doit être appelée en fonction du message de l’utilisateur plutôt que de l’assigner nous-mêmes.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Regardons maintenant la réponse et voyons comment elle est formatée :

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vous pouvez voir que le nom de la fonction est appelé et qu'à partir du message de l'utilisateur, le LLM a pu trouver les données correspondant aux arguments de la fonction.


## 3.Intégration des appels de fonction dans une application. 


Après avoir testé la réponse formatée du LLM, nous pouvons maintenant l'intégrer dans une application. 

### Gestion du flux 

Pour intégrer cela dans notre application, suivons les étapes suivantes : 

Tout d'abord, effectuons l'appel aux services OpenAI et stockons le message dans une variable appelée `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nous allons maintenant définir la fonction qui appellera l'API Microsoft Learn pour obtenir une liste de cours :


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Comme bonne pratique, nous verrons ensuite si le modèle souhaite appeler une fonction. Après cela, nous créerons l'une des fonctions disponibles et la ferons correspondre à la fonction qui est appelée.  
Nous prendrons ensuite les arguments de la fonction et les mapperons aux arguments provenant du LLM.

Enfin, nous ajouterons le message d'appel de fonction et les valeurs qui ont été retournées par le message `search_courses`. Cela donne au LLM toutes les informations dont il a besoin pour  
répondre à l'utilisateur en utilisant un langage naturel.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nous allons maintenant envoyer le message mis à jour au LLM afin de recevoir une réponse en langage naturel au lieu d'une réponse formatée en JSON d'API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Défi de code 

Excellent travail ! Pour continuer votre apprentissage de l'appel de fonction OpenAI, vous pouvez créer : https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Plus de paramètres de la fonction qui pourraient aider les apprenants à trouver plus de cours. Vous pouvez trouver les paramètres API disponibles ici :  
 - Créez un autre appel de fonction qui prend plus d'informations de l'apprenant, comme sa langue maternelle  
 - Créez une gestion des erreurs lorsque l'appel de fonction et/ou l'appel API ne renvoie aucun cours approprié


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant foi. Pour les informations critiques, une traduction professionnelle réalisée par un humain est recommandée. Nous déclinons toute responsabilité en cas de malentendus ou de mauvaises interprétations résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
